In [2]:
from azureml.core import Workspace, Experiment

# Corrected the below to run within a notebook actually on the studio.
# You guys designed this, so I don't understand why you didn't explain the
# various ways to run this, if it was your intent that I learn it.

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E653HCYGG to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-138422
Azure region: southcentralus
Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737
Resource group: aml-quickstarts-138422


In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute

### My compute name is always "ProjectOneAMLcompute" so when I kick this off I can confirm the test output at the last line of this cell.

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Looks like you've imported an unnecessary object, "ComputeTarget".
# You want a standard virtual machine, you got one.
# I don't see why you called it a "compute cluster" in line 3, if all you want is a CPU vm.
# The "max_nodes" call appears to be unnecessary, unless I'm wrong. ??? It would be nice to have office hours to discuss this stuff.

from azureml.core.runconfig import RunConfiguration
# You've already imported AmlCompute, so I'll leave that off, 
# but if "explicit is better than implicit" then the above should be replaced with the following line.
# from azureml.core.runconfig import ComputeTarget
# This exercise kinda proves that implicit code is harder to read. Did you guys do that on purpose?

# This part is for me, so I can see what I'm doing.
list_vms = AmlCompute.supported_vmsizes(workspace=ws)

###  uncomment to test to here...PRH this works 2/8
# print(list_vms)

### Here is my solution to specifying the vm_size variable ###
compute_config = RunConfiguration()
compute_config.amlcompute.vm_size = "STANDARD_D1_V2"

# I can't read your mind, so you should say if you want me to us a specific object.
# The above is a "git 'er done" method. The Microsoft tutorial shows a fairly complicated, but disaster-resistant
# block of code that would find or add a true "cluster"...And yes that actually does include reference to nodes.



# Here's the full monty from the Microsoft tutorial, with my comments and questions:

# compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', 'cpu-cluster')
# compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
# compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MAX_NODES', 4)

# The following uses CPU for virtual machine, but change "STANDARD_D2_V2" to "STANDARD_NC6" to get GPU.
# vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'STANDARD_D2_V2')

# The following code lines show a check for existing compute resources... Seems like a good idea to me.
# if compute_name in ws.compute_targets:
#     compute_target = ws.compute_targets[compute_name]
#     if compute_target and type(compute_target) is AmlCompute:
#         print('found compute target. just use it. '  + compute_name)
# else:
#     print('creating new compute target...')
#     provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes=compute_min_nodes, max_nodes=compute_max_nodes)

# creating the cluster, finally...
# compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

# This part I need to read up later:
# Microsoft tutorial reads, "can poll for a minimum number of nodes and for a specific timeout."
# and, "if no min node count is provided it will use the scale setting for the cluster"
# compute_target.wait_for_completion(show_output=True, mind_node_count=None, timeout_in_minutes=20)

# finally, looks like a best practice would be to add the print (below) line to keep status so we can see what's happening.
# For a more detailed view of current AmlCompute status, use get_status()
# print(compute_target.get_status().serialize())
# note that the above line will only work in conjunction with the rest of this block, see line 38 and/or line 46.

compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', 'ProjectOneAMLcompute')
print(compute_name)

# Question: this cell requires a workspace to run it. I have to start that workspace, and name it. That name is the output of the print command just above here.
# How does the compute_config = RunConfiguration() work, at the top? What's the name of the rsulting compute?
# Output should be "ProjectOneAMLcompute" See my notes top of this cell.

ProjectOneAMLcompute


In [12]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

print("The given code at least runs.")

# Specify parameter sampler
### YOUR CODE HERE ###
ps= RandomParameterSampling(  {
#    "learning_rate": normal(10, 3),
    "keep_probability": uniform(0.05, 0.1),
#    "batch_size": choice(16, 32, 64, 128)
    }
)
print("The microsoft docs are accurate enough...")

# Specify a Policy
# policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=100, slack_factor= 0.2, slack_amount= 0, delay_evaluation=0)


### The following code block was provided by Udacity ###
if "training" not in os.listdir():
    os.mkdir("./training")

print("next cell")


The given code at least runs.
The microsoft docs are accurate enough...
next cell


In [15]:
# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###

# PRH questions adn comments...
# SKLearn and estimators are deprecated, so I'm trying to use the current objects.
# This code has as a pre-requisite the train.py script, it appears. 
# What would it hurt for you guys to make a note of that here? This is training, right?
# Yeah it's obvious, but so what? My perception is that you don't actually want me to learn this, unless I learn it on my own. (Wait. What?)

from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory='.',
    script ="train.py",
    arguments=[dataset.as_named_input("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")],
    compute_target= compute_name, #from cell above
    environment=sklearn_env)



NameError: name 'dataset' is not defined

In [ ]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###


In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###